! nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd

## 패스 수정 부탁 드립니다.

In [2]:
import os
os.getcwd()

'/Users/ryleyun/Desktop/Github/Ryles_Practice/DACON'

In [3]:
### 데이터 불러오기 (정제 안 된 것)|
PATH = '..'
# PATH = '/content/drive/MyDrive/Github/Ryles_Practice'
train_pos = pd.read_csv(PATH + "/DATA/open/train_pos.csv")
test_pos = pd.read_csv(PATH + "/DATA/open/test_pos.csv")

In [4]:
train_pos = train_pos.fillna("없음")
test_pos = test_pos.fillna("없음")

In [5]:
train_name_key = [str(sent) for sent in train_pos['과제명'] + " " + train_pos['요약문_한글키워드']]
train_goal = [str(sent) for sent in train_pos['요약문_연구목표']]
train_cont = [str(sent) for sent in train_pos['요약문_연구내용']]
train_effect = [str(sent) for sent in train_pos['요약문_기대효과']]
test_name_key = [str(sent) for sent in test_pos['과제명'] + " " + test_pos['요약문_한글키워드']]
test_goal = [str(sent) for sent in test_pos['요약문_연구목표']]
test_cont = [str(sent) for sent in test_pos['요약문_연구내용']]
test_effect = [str(sent) for sent in test_pos['요약문_기대효과']]
train_label = [int(str(answer)) for answer in train_pos['label']]

In [6]:
train_concat = [str(name)+" "+str(key)+" "+" ".join(str(goal).split()[:100])+" "+" ".join(str(effect).split()[:100])+" "+" ".join(str(cont).split()[:100]) for name, key, goal, effect, cont in zip(train_pos['과제명'], train_pos['요약문_한글키워드'], train_pos['요약문_연구목표'], train_pos['요약문_기대효과'], train_pos['요약문_연구내용'])]
test_concat = [str(name)+" "+str(key)+" "+" ".join(str(goal).split()[:100])+" "+" ".join(str(effect).split()[:100])+" "+" ".join(str(cont).split()[:100]) for name, key, goal, effect, cont in zip(test_pos['과제명'], test_pos['요약문_한글키워드'], test_pos['요약문_연구목표'], test_pos['요약문_기대효과'], test_pos['요약문_연구내용'])]
train_label = train_pos['label']

In [7]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(train_name_key, train_label, test_size=0.1, random_state=42)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBRFClassifier

text_clf = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf", MultinomialNB())])
# text_clf = text_clf.fit(train_concat, train_label)

distribution_lr = loguniform(0.0001, 1.0)

params_clf = {"vect__ngram_range":[(1,2), (1,3), (1,5), (1,7)],
              "clf__alpha" : distribution_lr}

nb_clf = RandomizedSearchCV(text_clf, params_clf, n_iter=50, n_jobs=-1, verbose=2, cv=10, random_state = 42)
nb_clf = nb_clf.fit(train_data, train_label)

print("Best score: {}".format(nb_clf.best_score_))
nb_clf_best = nb_clf.best_estimator_

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf' , TfidfTransformer()),
                    ('clf' , BernoulliNB())])

distribution_bnb = loguniform(0.0001, 1.0)

params_clf = {"vect__ngram_range" : [(1,2),(1,3),(1,5),(1,7)],
                  "clf__alpha" : distribution_bnb,
                  "clf__binarize" : distribution_bnb,
              }

bn_clf = RandomizedSearchCV(text_clf, params_clf, n_iter = 50, cv = 10, random_state = 42, verbose = 2, n_jobs = -1)
bn_clf = bn_clf.fit(train_data, train_label)

print("Best score: {}".format(bn_clf.best_score_))
bn_clf_best = bn_clf.best_estimator_

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', XGBRFClassifier(learning_rate=0.01, subsample=0.9, random_state=42))])

params_clf = {"vect__ngram_range":[(1,2), (1,3), (1,5), (1,7)],
              "clf__n_estimators":[100, 200, 500],
              'clf__colsample_bytree':[0.3, 0.5, 0.9],
              'clf__max_depth':[3, 7, 10]}

rf_clf = RandomizedSearchCV(text_clf, params_clf, n_iter = 50, cv = 5, random_state = 42, verbose = 2, n_jobs = -1)
rf_clf = rf_clf.fit(train_data, train_label)

print("Best score: {}".format(rf_clf.best_score_))
rf_clf_best = rf_clf.best_estimator_

## 패스 수정 부탁 드립니다.

import pickle
# pickle.dump(nb_clf_best, open('../nb_clf_nk_model.sav', 'wb'))
# pickle.dump(bn_clf_best, open('../bn_clf_nk_model.sav', 'wb'))
pickle.dump(lr_clf_best, open('../lr_clf_nk_model.sav', 'wb'))

In [9]:
os.getcwd()

'/Users/ryleyun/Desktop/Github/Ryles_Practice/DACON'

!cp '/content/drive/MyDrive/Desktop/nb_clf_nk_model.sav'
!cp '/content/drive/MyDrive/Desktop.bn_clf_nk_model.sav'

In [10]:
import pickle
# nb_clf_best = pickle.load(open('nb_clf_nk_model.sav','rb'))
# bn_clf_best = pickle.load(open('bn_clf_nk_model.sav','rb'))
nb_clf_best = pickle.load(open('../../../nb_clf_nk_model.sav','rb'))
bn_clf_best = pickle.load(open('../../../bn_clf_nk_model.sav','rb'))
lgb_clf_best = pickle.load(open('../../../lgb_clf_best_model.sav','rb'))
xgb_clf_best = pickle.load(open('../../../xgb_clf_best_model.sav','rb'))

In [11]:
nb_clf_best_pred = nb_clf_best.predict(valid_data)
print('NaiveBaise')
print('잘못 분류된 개수: %d' % (valid_label != nb_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, nb_clf_best_pred))
bn_clf_best_pred = bn_clf_best.predict(valid_data)
print('BernoisBaise')
print('잘못 분류된 개수: %d' % (valid_label != bn_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, bn_clf_best_pred))
# rf_clf_best_pred = rf_clf_best.predict(valid_data)
# print('RF')
# print('잘못 분류된 개수: %d' % (valid_label != rf_clf_best_pred).sum())
# print('정확도: %.3f' % accuracy_score(valid_label, rf_clf_best_pred))
lgb_clf_best_pred = lgb_clf_best.predict(valid_data)
print('LGB')
print('잘못 분류된 개수: %d' % (valid_label != lgb_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, lgb_clf_best_pred))
xgb_clf_best_pred = xgb_clf_best.predict(valid_data)
print('XGB')
print('잘못 분류된 개수: %d' % (valid_label != xgb_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, xgb_clf_best_pred))

NaiveBaise
잘못 분류된 개수: 1251
정확도: 0.928
BernoisBaise
잘못 분류된 개수: 1327
정확도: 0.924
LGB
잘못 분류된 개수: 601
정확도: 0.966
XGB
잘못 분류된 개수: 610
정확도: 0.965


In [12]:
from sklearn.ensemble import StackingClassifier

stack = StackingClassifier(estimators = [('nb', nb_clf_best), ('bn', bn_clf_best), ('lgb', lgb_clf_best), ('xgb', xgb_clf_best)], n_jobs=5)

stack.fit(train_data, train_label)

[18:08:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "sample_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGSEGV(-11)}

## 패스 수정 부탁 드립니다.

In [ ]:
pred = stack.predict(test_name_key)

In [ ]:
sum(pred!=0)

In [ ]:
sample_submission=pd.read_csv('../DATA/open/sample_submission.csv')
sample_submission['label']=pred
sample_submission.to_csv('../DATA/Result/NB_BN_LGB_XGB_stack.csv', index=False)